## Step 16 복잡한 계산 그래프(구현 편)

In [55]:
import numpy as np

In [56]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        self.data = data
        self.grad = None 
        self.creator = None
        self.generation = 0 #세대 수를 기록하는 변수 추가

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #세대를 기록한다(부모 세대 +1)

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) 

        funcs = []# DeZero함수를 리스트에 추가하는 add_func함수 만듬
        seen_set = set() # 같은 함수 중복 막기위해 집합으로

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)#

        while funcs:
            f = funcs.pop() 
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None: 
                    x.grad = gx
                else:
                    x.grad = x.grad + gx 

                if x.creator is not None:
                    add_func(x.creator)# 수정전 funcs.append(x.creator)
    
    def cleargrad(self):
        self.grad = None 

In [57]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [58]:
class Function:
    def __call__(self, *inputs): 
        xs = [x.data for x in inputs] 
        ys = self.forward(*xs) 
        if not isinstance(ys, tuple): 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] 

        self.generation = max([x.generation for x in inputs]) #fuction클래스 generation은 입력변수와 같은 값, 입력변수 많으면 generation 가장 큰 값으로 선택

        for output in outputs:
            output.set_creator(self) 
        self.inputs = inputs
        self.outputs = outputs 
        return outputs if len(outputs) >1 else outputs[0] 
    
    def forward(self, xs):
        raise NotImplementedError()
    def backward(self, gys):
        raise NotImplementedError()

In [59]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
def square(x):
    return Square()(x) 
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
def exp(x):
    return Exp()(x)

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy 
def add(x0, x1):
    return Add()(x0, x1)

In [61]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
